# Explanation

This code, which implements the automatic generation of effective operators in the effective field theory of a single scalar field with $\phi \leftrightarrow - \phi$ symmetry, establishes a framework for approaching the case of ultimate interest, where we seek to generate higher dimensional operators in more complicated EFTs such as Standard Model Effective Field Theory (SMEFT). In particular, a general strategy for generation of operators, and reduction of the operator basis through integration by parts (IBP) and equations of motion (EOM), which may be generalized to more complicated cases, is developed. 

# Define Classes for Field and Term Objects

Define classes for field objects and term objects, which are monomials built from contracted fields. Derivatives are a particular type of field object.

In [25]:
import numpy as np

class field(object):
    def __init__(self, symbol, massDim, index=None):
        self.symbol = symbol #string symbol for field
        self.index = index #string indicating type of index - here, only Lorentz
        self.massDim= massDim #list of connected terms, includes repeats
    def info(self):
        return 'symbol: ' + str(self.symbol) + '\n' \
            +   'massDim: ' + str(self.massDim) + '\n' \
            +   'index: ' + str(self.index)
    def get_symbol(self):
        return self.symbol
    def get_index(self):
        return self.index
    def get_massDim(self):
        return int(self.massDim)
    def __eq__(self, other):
        eq = (self.symbol == other.symbol)
        return eq

In [26]:
class term(object):
    def __init__(self, field_list, contractions=[]):
        self.field_list = field_list #ordered list of field objects
        self.contractions = contractions #list of 2-tuples, indicating index values of fields in self.fields that are contracted 
    def __eq__(self, other):
        eq = (self.field_list == other.field_list)&(self.contractions == other.contractions)
        return eq
    def __str__(self):
        return 'fields: ' + str([item.get_symbol() for item in self.field_list]) + '\n' \
            + 'contractions: ' + str(self.contractions)
    def get_field_list(self):
        return self.field_list
    def get_contractions(self):
        return self.contractions
    def set_field_list(self, field_list):
        self.field_list = field_list
    def set_contractions(self, contractions):
        self.contractions = contractions
    def massDim(self):
        massDim = 0
        field_list = self.field_list
        #print(field_list)
        for field in field_list:
            massDim += field.get_massDim()
        return massDim 
    def get_field_symbols(self):
        field_symbols = []
        for item in self.field_list:
            field_symbols.append(item.get_symbol())
        return field_symbols       
    

As an example, consider the term $(\partial_{\mu} \phi)(\partial^{\mu} \phi)$, built from multiplication/contraction of the fields $\phi$ and $\partial$:

In [29]:
P = field('P', 1)
D = field('D', 1, 'u')
A = term([D, P, D, P], [(0, 2)])
print(A)

fields: ['D', 'P', 'D', 'P']
contractions: [(0, 2)]


# Generate All Terms of a Given Mass Dimension

The main function used to generate all terms of a given mass dimension (prior to reduction via integration by parts and equations of motion) is called *generate_contracted_terms(massDim, num_free_indices)*, which generates all inequivalent terms (modulo commutation of derivatives and fields) of a given mass dimension with a specified number of free indices (in the simple case of the scalar field, these are only Lorentz indices). This will serve two goals:

- We all fully contracted (no free indices), inequivalent terms up to a given mass dimension. We can get this by calculating the terms for each mass dimension separately. This is what *generate_contracted_terms(massDim, num_free_indices)* does.
- In generating the integration by parts (IBP) linear relations among operators of mass dimension $M$, there will be one relation for each term of dimension $M-1$ with one free Lorentz index. Thus, setting *num_free_indices = 1* and *massDim = M-1* in *generate_contracted_terms(massDim, num_free_indices)* enables us to generate all of the IBP relations among operators of mass dimension $M$. See the section "Generate IBP Relations" below for more detail.

## *Enumerate ways of distributing derivatives among fields in uncontracted terms.*

Initially, ignore the structure of the contractions and focus on uncontracted terms (all indices free). Uncontracted terms are differentiated by the number of derivatives associated on each field. Terms with the same number of fields that differ in the number of derivatives acting on the fields - assume for each term that the list of number of derivatives acting on each field, num_derivs_list, has been sorted - are inequivalent. Thus, for a given mass dimension M, uncontracted terms can be partitioned according to $(n, m)$, where $n$ is the number of derivatives, $m$ is the number of fields, $n + m = M$, and $m >= 1$. For uncontracted terms associated with each pair $(n, m)$, we can further partition the set into sorted lists $(i_1, ... , i_m)$ of length m indicating the number of derivatives acting on each field. Uncontracted terms are inequivalent if their sorted lists $(i_1, ... , i_m)$ differ. The function *sums_to_n(n, m)* lists all ways of summing $m$ positive integers to give $n$; *sums_to_n_padded(n, m)* lists all ways of summing $m$ non-negative integers (including zero) to give $n$. The functions *non_increasing(L)* and *pad_w_zeros(list_of_lists, size)* are helper functions for *sums_to_n_padded(n, m)*, which gives all inequivalent ways of distributing uncontracted derivatives among fields. 

In [18]:
def non_increasing(L):
    return all(x>=y for x, y in zip(L, L[1:]))

def sums_to_n(n, m): # generates partitions of derivatives to each field
    #INPUT
    #n: number we want to sum to; i.e. total number of derivatives
    #m: number of digits in sum; i.e. total number of fields
    #OUTPUT
    #list of lists of digits, with each sublist no longer than m
    #EXPLANATION
    #list of combos for n can be generated from list of combos for n-1 either by appending 1 to each of the combos
    #for n-1, or adding 1 to just one of the elements in each combo. To ensure no repeats, we require the lists to be
    #non-increasing.
    combos_list = []
    if n==0:
        combo = m*[0]
        combos_list.append(combo)
        return combos_list
    elif n==1:
        combo = [1] + (m-1)*[0]
        combos_list.append(combo)
        return combos_list
    else:
        combos_list_prev = sums_to_n(n-1, n-1)
        for combo in combos_list_prev:
            combo_new1 = combo + [1]
            if non_increasing(combo_new1) and combo_new1 not in combos_list:
                combos_list.append(combo_new1) 
            for i in range(len(combo)):
                combo_new2 = combo.copy()
                combo_new2[i] += 1
                if non_increasing(combo_new2) and combo_new2 not in combos_list:
                    combos_list.append(combo_new2) 
        combos_list_trunc = [combo for combo in combos_list if len(combo) <= m]    
        return combos_list_trunc

def pad_w_zeros(list_of_lists, size):
    #INPUT
    #list_of_lists: list of lists of numbers; all sublists should have length less than or equal to size
    #size: int, common size of all lists after padding
    #OUTPUT
    #list_of_lists_padded: padded list of lists, where each sublist has length size
    
    if size < max([len(sublist) for sublist in list_of_lists]):
        print("'size' input must be larger than or equal max length of sublists in 'list_of_lists' input.")
        return None
    else:
        list_of_lists_padded = []
        for sublist in list_of_lists:
            num_zeros = size - len(sublist)
            sublist_padded = sublist + num_zeros*[0]
            list_of_lists_padded.append(sublist_padded)
        return list_of_lists_padded
        
def sums_to_n_padded(n, m):
    return pad_w_zeros(sums_to_n(n, m), m)

In [21]:
sums_to_n(7, 4)

[[4, 1, 1, 1],
 [3, 2, 1, 1],
 [2, 2, 2, 1],
 [5, 1, 1],
 [4, 2, 1],
 [3, 3, 1],
 [3, 2, 2],
 [6, 1],
 [5, 2],
 [4, 3],
 [7]]

In [19]:
sums_to_n_padded(7,4)

[[4, 1, 1, 1],
 [3, 2, 1, 1],
 [2, 2, 2, 1],
 [5, 1, 1, 0],
 [4, 2, 1, 0],
 [3, 3, 1, 0],
 [3, 2, 2, 0],
 [6, 1, 0, 0],
 [5, 2, 0, 0],
 [4, 3, 0, 0],
 [7, 0, 0, 0]]

## *Generate terms with different contractions*

Next we want to differentiate between contracted terms that may have the same number of fields and number of derivatives per field, but differ by virtue of their contractions. That is, we want to generate all terms with a given *num_derivs_list* that are inequivalent by virtue of having inequivalent contractions. Here we work with "reduced" contractions, in which the indices in each 2-tuple indicate the index of the fields on which the contracted derivatives act. This can be inferred, but is distinct, from the format of contractions used to specify a term object, in which the contracted indices designate the place of a derivative in the field list used to define the term. 

In [35]:
def equiv_contraction(contraction1, contraction2, num_derivs_list):
    #INPUT
    #contraction1: 2-tuple of integers designating reduced contraction
    #contraction2: 2-tuple of integers designating reduced contraction
    #num_derivs_list: number of derivatives per field, assumed shared by term1 and term2
    #OUTPUT
    #equiv: bool indicating whether two contractions are the same with respect to num_derivs_list
    indices_same_1 = contraction1[0]==contraction1[1]
    indices_same_2 = contraction2[0]==contraction2[1]
    #check whether number of derivatives associated with indices are the same.
    equiv = (num_derivs_list[contraction1[0]] == num_derivs_list[contraction2[0]] and num_derivs_list[contraction1[1]] == num_derivs_list[contraction2[1]]) or (num_derivs_list[contraction1[0]] == num_derivs_list[contraction2[1]] and num_derivs_list[contraction1[1]] == num_derivs_list[contraction2[0]])
    #check whether indices of each contraction are on same field or different fields for each contraction.
    #contractions are only equivalent if they are either both on the same field or both on different fields.
    equiv = equiv and (indices_same_1 == indices_same_2)
    return equiv

def equiv_contraction_list(contraction_list1, contraction_list2, num_derivs_list):
    #INPUT
    #contraction_list1: list of reduced contractions of term1
    #contraction_list2: list of reduced contractions of term2
    #num_derivs_list: number of derivatives per field, assumed shared by term1 and term2
    #OUTPUT
    #True if the contraction lists contain equivalent contractions, False otherwise
    #EXPLANATION
    #returns True if the two lists of contractions are equivalent relative to num_derivs_list, and False otherwise.
    #for each contraction in contraction_list1, see if it is equivalent to one of the remaining contractions
    #in contraction_list2. remove an element from contraction_list2 if it is found to be equivalent to one from 
    #contraction_list1. for each element of contraction_list1, we must find an equivalent element of 
    #contraction_list2.
    if len(contraction_list1) != len(contraction_list2):
        return False
    contraction_list1_copy = contraction_list1.copy() #make copies to avoid changing lists
    contraction_list2_copy = contraction_list2.copy()
    for contraction1 in contraction_list1_copy:
        match_found = False
        for contraction2 in contraction_list2_copy:
            if equiv_contraction(contraction1, contraction2, num_derivs_list):
                match_found = True
                contraction_list2_copy.remove(contraction2)
                break
        #if after looping through all contraction_list2 no match is found, return False
        if match_found == False:
            return False            
    return True #should be true if match_found is true for all elements

def contained_in(contraction_list, list_of_contraction_lists, num_derivs_list): 
    #INPUT
    #contraction_list: list of 2-tuples indicating contractions
    #list_of_contraction_lists: list of contraction lists
    #num_derivs_list: list of integers indicating number of derivatives acting on each field
    #EXPLANATION
    #determines whether contraction_list is contained in list_of_contraction_lists - i.e., equivalent to one
    #of the contraction lists in list_of_contraction_lists. 
    for i in range(len(list_of_contraction_lists)):
        if equiv_contraction_list(contraction_list, list_of_contraction_lists[i], num_derivs_list):
            return True
        else:
            pass
    return False

def generate_contractions(num_derivs_list, num_contractions):
    #INPUT
    #num_derivs_list: list where each list element corresponds to a single field and each element to the number
    #of derivatives acting on each field
    #num_contractions: integer indicating number of contractions in term, can be no larger than half the total number
    #of derivatives, equal to the sum of elements in partition. 
    #OUTPUT
    #contractions_list: list of lists of reduced contractions, e.g. for num_derivs_list = [4, 3], num_contractions = 3, one element would be [(0, 0),(0, 0),(1, 1)]
    #another would be [(0, 1),(0, 1),(0, 0)].
    #for num_derivs_list = [3, 2, 2] and num_contractions = 3, [(0, 0), (0, 1), (1, 2)] (one 2 left free) and [(0, 1), (1, 2), (0, 2)] (one 0 left free)
    #EXPLANATION
    #Generates all reduced contractions for given num_derivs_list and num_contractions. Use recursion.  
    if num_contractions == 0: 
        contractions_list = [[]] # list of lists. each sub-list is of length num_contractions.
        return contractions_list
    else: 
        contractions_list = []
        uncontracted = num_derivs_list
        N_fields = len(num_derivs_list)
        #loop over all possible values of first contraction; add these to list of contractions produced by generate_contractions(num_derivs_list_red, num_contractions - 1)
        for i1 in range(N_fields):
            for i2 in range(N_fields):
                if i1 != i2:
                    if uncontracted[i1] > 0 and uncontracted[i2] > 0:
                        uncontracted_new = uncontracted.copy()
                        contraction = (i1, i2)
                        uncontracted_new[i1] -= 1
                        uncontracted_new[i2] -= 1
                        contractions_list_sub = generate_contractions(uncontracted_new, num_contractions - 1)
                        for sublist in contractions_list_sub:
                            sublist_new = [contraction] + sublist
                            #print("sublist_new: " + str(sublist_new))
                            contractions_list.append(sublist_new)
                    else:
                        pass
                elif i1 == i2:
                    if uncontracted[i1] - 1 > 0: #must be at least 2 in uncontracted[i1]
                        uncontracted_new = uncontracted.copy()
                        contraction = (i1, i1)
                        uncontracted_new[i1] -= 2
                        contractions_list_sub = generate_contractions(uncontracted_new, num_contractions-1)
                        for sublist in contractions_list_sub:
                            sublist_new = [contraction] + sublist
                            contractions_list.append(sublist_new)
                    else:
                        pass
        return contractions_list  
    
def reduce_contractions(list_of_contraction_lists, num_derivs_list):
    #INPUT
    #contractions_list: list of lists of contractions
    #num_derivs_list: list of number of derivatives attached to each field
    #OUTPUT
    #contractions_unique: list of lists of contractions
    #EXPLANATION
    #returns list of unique contraction lists in list_of_contraction_lists
    contractions_unique = []
    for i in range(len(list_of_contraction_lists)):
        if not contained_in(list_of_contraction_lists[i], contractions_unique, num_derivs_list):
            contractions_unique.append(list_of_contraction_lists[i])
        else:
            pass
    return contractions_unique

def generate_unique_contractions(num_derivs_list, num_contractions):
    #INPUT
    #num_derivs_list: list of number of derivatives acting on each field
    #num_contractions: number of contractions
    #OUTPUT
    #contractions_unique: list of distinct lists of contractions, where each contraction list is associated to
    #a distinct term.
    #EXPLANATION
    #generates a list of lists of contractions, such that no list of contractions is equivalent to any other
    list_of_contraction_lists = generate_contractions(num_derivs_list, num_contractions)
    contractions_unique = reduce_contractions(list_of_contraction_lists, num_derivs_list)
    return contractions_unique
        

In [36]:
num_derivs_list = [5,4]
num_contractions = 4
generate_unique_contractions(num_derivs_list, num_contractions)

[[(0, 0), (0, 0), (0, 1), (1, 1)],
 [(0, 0), (0, 0), (1, 1), (1, 1)],
 [(0, 0), (0, 1), (0, 1), (0, 1)],
 [(0, 0), (0, 1), (0, 1), (1, 1)],
 [(0, 1), (0, 1), (0, 1), (0, 1)]]

This corresponds to (with $\nu$ the one free index, and the $\mu_i$ contracted):

$(\partial_{\mu_1} \partial^{\mu_1} \partial_{\mu_2} \partial^{\mu_2} \partial_{\mu_3} \phi)( \partial^{\mu_3} \partial_{\mu_4} \partial^{\mu_4} \partial^{\nu} \phi)$,

$(\partial_{\mu_1} \partial^{\mu_1} \partial_{\mu_2} \partial^{\mu_2} \partial^{\nu} \phi) ( \partial_{\mu_3} \partial^{\mu_3} \partial_{\mu_4} \partial^{\mu_4} \phi)$,

$(\partial_{\mu_1} \partial^{\mu_1} \partial_{\mu_2} \partial_{\mu_3} \partial_{\mu_4} \phi) ( \partial^{\mu_2} \partial^{\mu_3} \partial^{\mu_4} \partial^{\nu} \phi)$,

$(\partial_{\mu_1} \partial^{\mu_1} \partial_{\mu_2} \partial_{\mu_3} \partial^{\nu} \phi) ( \partial^{\mu_2} \partial^{\mu_3} \partial_{\mu_4} \partial^{\mu_4} \phi)$,

$(\partial_{\mu_1} \partial_{\mu_2} \partial_{\mu_3} \partial_{\mu_4} \partial^{\nu} \phi) ( \partial^{\mu_1} \partial^{\mu_2} \partial^{\mu_3} \partial^{\mu_4} \phi)$




In [1]:
def convert_to_term_object(num_derivs_list, contraction_list):
    #INPUT
    #num_derivs_list: list of integers indicating the number of derivatives acting on each field
    #contraction_list: list of 2-tuples of integers indicating which derivatives are contracted; more precisely,
    #integers in 2-tuples indicate which field the contracted derivative acts on. 
    #OUTPUT
    #term_object: a term object with the specified number of fields and number of derivatives acting on each field, 
    #and derivatives contracted as per contraction_list
    
    num_phi = len(num_derivs_list)
    num_contractions = len(contraction_list)
    field_list = []
    contraction_transform_list = []
    
    #build field_list
    for i in range(num_phi):
        P_i = field('P', 1)
        for j in range(num_derivs_list[i]):
            D_ij = field('D', 1, 'u') #u for Lorentz index
            field_list.append(D_ij)
        field_list.append(P_i)
    
    counts = num_phi*[0] #stores number of contractions into phi field place
    #build contraction_list
    for i in range(num_contractions):
        contraction = contraction_list[i]
        phi1 = contraction[0]
        phi2 = contraction[1]
        
        index1 = sum(num_derivs_list[:phi1]) + phi1 + counts[phi1]
        counts[phi1] += 1
        index2 = sum(num_derivs_list[:phi2]) + phi2 + counts[phi2]
        counts[phi2] += 1
        
        contraction_transform = (index1, index2)
        contraction_transform_list.append(contraction_transform)
           
    #instantiate term object
    term_object = term(field_list, contraction_transform_list)
            
    return term_object

In [2]:
num_derivs_list = [3, 2, 1, 1]
contraction_list = [(0, 1), (2, 3)]

term_object = convert_to_term_object(num_derivs_list, contraction_list)
print(term_object.get_contractions())
print([item.get_symbol() for item in term_object.get_field_list()])

NameError: name 'field' is not defined

In [10]:
#generate terms so that free indices are allowed. 
def generate_contracted_terms(massDim, num_free_indices):
    #INPUT
    #massDim: int, mass dimension of terms to be generated.
    #OUTPUT
    #num_free_indices: number of free Lorentz indices we want to leave open
    #EXPLANATION
    #num_free_indices = num_derivs - 2*num_contractions, so 
    #num_contractions = .5*(num_derivs - num_free_indices), so num_derivs has to be a multiple of 2 greater than 
    #num_free_indices
    M = massDim
    contracted = []
    #print("M: " + str(M))
    for i in range(int(M)): #loop over number of derivatives in term
        num_derivs = i #need number of derivatives to be even to ensure that every index is contracted
        num_fields = M - num_derivs
        #print("")
        #print("")
        #print("in generate_fully_contracted, (num_derivs, num_fields):" + str((num_derivs, num_fields)))
        list_of_num_derivs_lists = sums_to_n_padded(num_derivs, num_fields) #list of lists, with each sublist containing a different distribution of derivatives among fields. 
        
        #check that num_derivs allows for specified number of free indices, if not, go to next i in loop. 
        if (num_derivs - num_free_indices)%2 == 0: 
            num_contractions = .5*(num_derivs - num_free_indices)
        else: #if number of derivatives does not allow for specified number of free indices, then move to next number
            #print("continue")
            continue 
        
        for j in range(len(list_of_num_derivs_lists)): #loop over different ways of distributing derivatives among fields
            num_derivs_list = list_of_num_derivs_lists[j]
            #print("")
            #print("num_derivs_list: " + str(num_derivs_list)) 
            list_of_contraction_lists = generate_unique_contractions(num_derivs_list, num_contractions)
            for contraction_list in list_of_contraction_lists: #loop over inequivalent ways of contracting Lorentz indices of derivatives 
                #print("")
                #print("contraction_list: " + str(contraction_list))
                term_object = convert_to_term_object(num_derivs_list, contraction_list)
                #print("term_object.get_field_symbols(): " + str(term_object.get_field_symbols()))
                #check if number of P's, equal to len(num_derivs_list), is even. this enforces P<->-P symmetry
                num_phi = len([item for item in term_object.get_field_list() if item.get_symbol()=='P'])
                if num_phi%2 == 0:
                    contracted.append(term_object)        
    return contracted


In [11]:
#generate terms w/ no free indices.
def generate_fully_contracted(massDim):
    #INPUT
    #massDim: int, mass dimension of terms to be generated.
    #OUTPUT
    #fully_contracted: a list of all term objects of mass dimension massDim, all fully contracted. 
    fully_contracted = generate_contracted_terms(massDim, 0)     
    return fully_contracted

Check that no terms with odd number of phi fields are generated.

In [12]:
massDim = 5
num_free_indices = 0
terms = generate_contracted_terms(massDim, num_free_indices)
print("terms: " + str(terms))
for item in terms:
    print("term_object.get_field_symbols(): " + str(item.get_field_symbols()))
    print("term_object.get_contractions(): " + str(item.get_contractions()))
terms = generate_fully_contracted(massDim)
print("terms: " + str(terms))
for item in terms:
    print("term_object.get_field_symbols(): " + str(item.get_field_symbols()))
    print("term_object.get_contractions(): " + str(item.get_contractions()))

terms: []
terms: []


Check that generate_contracted_terms(massDim, num_free_indices=0) gives same results as generate_fully_contracted(massDim)

In [13]:
massDim = 8
num_free_indices = 0
terms = generate_contracted_terms(massDim, num_free_indices)

for item in terms:
    print("term_object.get_field_symbols(): " + str(item.get_field_symbols()))
    print("term_object.get_contractions(): " + str(item.get_contractions()))


term_object.get_field_symbols(): ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P']
term_object.get_contractions(): []
term_object.get_field_symbols(): ['D', 'P', 'D', 'P', 'P', 'P', 'P', 'P']
term_object.get_contractions(): [(0, 2)]
term_object.get_field_symbols(): ['D', 'D', 'P', 'P', 'P', 'P', 'P', 'P']
term_object.get_contractions(): [(0, 1)]
term_object.get_field_symbols(): ['D', 'P', 'D', 'P', 'D', 'P', 'D', 'P']
term_object.get_contractions(): [(0, 2), (4, 6)]
term_object.get_field_symbols(): ['D', 'D', 'P', 'D', 'P', 'D', 'P', 'P']
term_object.get_contractions(): [(0, 1), (3, 5)]
term_object.get_field_symbols(): ['D', 'D', 'P', 'D', 'P', 'D', 'P', 'P']
term_object.get_contractions(): [(0, 3), (1, 5)]
term_object.get_field_symbols(): ['D', 'D', 'D', 'P', 'D', 'P', 'P', 'P']
term_object.get_contractions(): [(0, 1), (2, 4)]
term_object.get_field_symbols(): ['D', 'D', 'P', 'D', 'D', 'P', 'P', 'P']
term_object.get_contractions(): [(0, 1), (3, 4)]
term_object.get_field_symbols(): ['D', 'D', 'P

Appears to give same result. Check 

Generate all terms of massDim 8. Reduce basis according to integration by parts. 

In [14]:
massDim = 8
num_free_indices = 0
terms = generate_contracted_terms(massDim, num_free_indices)

for item in terms:
    print("term_object.get_field_symbols(): " + str(item.get_field_symbols()))
    print("term_object.get_contractions(): " + str(item.get_contractions()))

term_object.get_field_symbols(): ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P']
term_object.get_contractions(): []
term_object.get_field_symbols(): ['D', 'P', 'D', 'P', 'P', 'P', 'P', 'P']
term_object.get_contractions(): [(0, 2)]
term_object.get_field_symbols(): ['D', 'D', 'P', 'P', 'P', 'P', 'P', 'P']
term_object.get_contractions(): [(0, 1)]
term_object.get_field_symbols(): ['D', 'P', 'D', 'P', 'D', 'P', 'D', 'P']
term_object.get_contractions(): [(0, 2), (4, 6)]
term_object.get_field_symbols(): ['D', 'D', 'P', 'D', 'P', 'D', 'P', 'P']
term_object.get_contractions(): [(0, 1), (3, 5)]
term_object.get_field_symbols(): ['D', 'D', 'P', 'D', 'P', 'D', 'P', 'P']
term_object.get_contractions(): [(0, 3), (1, 5)]
term_object.get_field_symbols(): ['D', 'D', 'D', 'P', 'D', 'P', 'P', 'P']
term_object.get_contractions(): [(0, 1), (2, 4)]
term_object.get_field_symbols(): ['D', 'D', 'P', 'D', 'D', 'P', 'P', 'P']
term_object.get_contractions(): [(0, 1), (3, 4)]
term_object.get_field_symbols(): ['D', 'D', 'P

Implement integration by parts. To each of the above terms there corresponds a different linear integration by parts (IBP) linear dependence relation. Ultimately, we want to see how many of these relations are linearly independent, to determine the number of independent operators of a given mass dimension. 

In [15]:
massDim = 7
num_free_indices = 1
terms = generate_contracted_terms(massDim, num_free_indices)

for item in terms:
    print("term_object.get_field_symbols(): " + str(item.get_field_symbols()))
    print("term_object.get_contractions(): " + str(item.get_contractions()))

term_object.get_field_symbols(): ['D', 'P', 'P', 'P', 'P', 'P', 'P']
term_object.get_contractions(): []
term_object.get_field_symbols(): ['D', 'P', 'D', 'P', 'D', 'P', 'P']
term_object.get_contractions(): [(0, 2)]
term_object.get_field_symbols(): ['D', 'D', 'P', 'D', 'P', 'P', 'P']
term_object.get_contractions(): [(0, 1)]
term_object.get_field_symbols(): ['D', 'D', 'P', 'D', 'P', 'P', 'P']
term_object.get_contractions(): [(0, 3)]
term_object.get_field_symbols(): ['D', 'D', 'D', 'P', 'P', 'P', 'P']
term_object.get_contractions(): [(0, 1)]
term_object.get_field_symbols(): ['D', 'D', 'D', 'D', 'P', 'D', 'P']
term_object.get_contractions(): [(0, 1), (2, 3)]
term_object.get_field_symbols(): ['D', 'D', 'D', 'D', 'P', 'D', 'P']
term_object.get_contractions(): [(0, 1), (2, 5)]
term_object.get_field_symbols(): ['D', 'D', 'D', 'P', 'D', 'D', 'P']
term_object.get_contractions(): [(0, 1), (2, 4)]
term_object.get_field_symbols(): ['D', 'D', 'D', 'P', 'D', 'D', 'P']
term_object.get_contractions(): [

In [16]:
massDim = 5
num_free_indices = 1
terms1 = generate_contracted_terms(massDim, num_free_indices)

for item in terms1:
    print("term_object.get_field_symbols(): " + str(item.get_field_symbols()))
    print("term_object.get_contractions(): " + str(item.get_contractions()))

term_object.get_field_symbols(): ['D', 'P', 'P', 'P', 'P']
term_object.get_contractions(): []
term_object.get_field_symbols(): ['D', 'D', 'P', 'D', 'P']
term_object.get_contractions(): [(0, 1)]
term_object.get_field_symbols(): ['D', 'D', 'P', 'D', 'P']
term_object.get_contractions(): [(0, 3)]
term_object.get_field_symbols(): ['D', 'D', 'D', 'P', 'P']
term_object.get_contractions(): [(0, 1)]


In [17]:
massDim = 6
num_free_indices = 0
terms2 = generate_contracted_terms(massDim, num_free_indices)

for item in terms2:
    print("term_object.get_field_symbols(): " + str(item.get_field_symbols()))
    print("term_object.get_contractions(): " + str(item.get_contractions()))

term_object.get_field_symbols(): ['P', 'P', 'P', 'P', 'P', 'P']
term_object.get_contractions(): []
term_object.get_field_symbols(): ['D', 'P', 'D', 'P', 'P', 'P']
term_object.get_contractions(): [(0, 2)]
term_object.get_field_symbols(): ['D', 'D', 'P', 'P', 'P', 'P']
term_object.get_contractions(): [(0, 1)]
term_object.get_field_symbols(): ['D', 'D', 'D', 'P', 'D', 'P']
term_object.get_contractions(): [(0, 1), (2, 4)]
term_object.get_field_symbols(): ['D', 'D', 'P', 'D', 'D', 'P']
term_object.get_contractions(): [(0, 1), (3, 4)]
term_object.get_field_symbols(): ['D', 'D', 'P', 'D', 'D', 'P']
term_object.get_contractions(): [(0, 3), (1, 4)]
term_object.get_field_symbols(): ['D', 'D', 'D', 'D', 'P', 'P']
term_object.get_contractions(): [(0, 1), (2, 3)]


# Reduce Operator Basis with Integration by Parts (IBP) and Equation of Motion (EOM)

IBP and EOM can be used to reduced the number of independent operators up to a given mass dimension. First focus on IBP, then EOM.  

## Generate IBP Relations

Every IBP relation arises from a relation $\int d^{4}x \ \partial_{\mu} T^{\mu} = 0$, where $T^{\mu}$ is a term with one free Lorentz index. There will be one IBP relation for each $T^{\mu}$. If $T^{\mu}$ contains $n$ $\phi$ fields, by the product rule of differentiation the IBP relation will consist of $n$ terms summing to zero. All terms in a single IBP relation have the same mass dimension; thus IBP relations do not mix mass dimensions, while EOM relations do.

In [18]:
def differentiate(term_object):
    #INPUT
    #term_object: term object with one free index
    #OUTPUT
    #term_list: list of term objects in sum resulting from differentiation
    #EXPLANATION
    #implements product rule on a term of dim M-1 to generate a list of terms of dim M. if there are N P fields 
    #in a term, there will be N terms in the product rule expansion. 
    #TO DO: check - test for correct number of indices. 
    
    field_list = term_object.get_field_list()
    contraction_list = term_object.get_contractions()
    
    L = len(field_list)
    term_list = [] #store resulting terms
    
    #set field lists for N new terms of dim M. 
    new_field_lists = [] #stores field lists for generated terms
    for i in range(L):
        D = field('D', 1, 'u')
        if field_list[i].get_symbol() == 'P':
            #print("field loop, i = " + str(i))
            #insert D before P
            field_list_new = field_list.copy()
            field_list_new.insert(i, D) #insert derivative D into field list right before P
            new_field_lists.append(field_list_new)
    
    #print("new_field_lists: " + str(len(new_field_lists)))
    #set contraction lists for N new terms of dim M. 
    #EXPLANATION: find uncontracted index in term_object. then, looping through contractions in contraction_list,
    #1) if index comes before index of inserted D, leave the same; if index comes after index of inserted D, increment
    #index by 1. 
    #2) add contraction between index of inserted D and free index.  
    new_contraction_lists = []
    
    #find uncontracted index in term_object, since this will be contracted with external derivative
    contracted_indices = [] #store contracted indices to find the one free index
    #free_index = None
    for contraction in contraction_list:
        contracted_indices.append(contraction[0])
        contracted_indices.append(contraction[1])
    #print("contracted_indices: " + str(contracted_indices))
    free_index = 9999 #set to default value not equal to one of the expected possibilities
    for i in range(len(field_list)): 
        if field_list[i].get_symbol() == 'D' and i not in contracted_indices:
            free_index = i
            #print("free_index: " + str(free_index))
        else:
            pass
            #print("No free index found.")
    
    for i in range(L): #for each P field, there is a separate new term, with its own new contraction_list, contraction_list_new
        if field_list[i].get_symbol() == 'P': #create new contraction list for each P
            #print("contraction loop, i = " + str(i))
            contraction_list_new = []
            #1) if index comes before index of inserted D, leave the same; if index comes after index of inserted D, increment
            #index by 1.
            for contraction in contraction_list:
                contraction_new = (None, None)
                if contraction[0] < i:
                    contraction_new_0 = contraction[0]
                elif contraction[0] > i:
                    contraction_new_0 = contraction[0] + 1
                elif contraction[0] == i:
                    print("Error: Cannot contract into P field.")

                if contraction[1] < i:
                    contraction_new_1 = contraction[1]
                elif contraction[1] > i:
                    contraction_new_1 = contraction[1] + 1
                elif contraction[1] == i:
                    print("Error: Cannot contract into P field.")
                
                contraction_new = (contraction_new_0, contraction_new_1)
                contraction_list_new.append(contraction_new)    
        
            #2) add contraction between index of inserted D and free index.
            #find free index
            #also shift free index
            #free_index_new = None
            if free_index < i:
                free_index_new = free_index
            elif free_index > i:
                free_index_new = free_index + 1
            elif free_index == i:
                print("free_index == i")
                print("Error: Cannot contract into P field.")

            if free_index < i: 
                contraction_list_new.append((free_index_new, i))
            elif free_index > i:
                contraction_list_new.append((i, free_index_new))
            
            new_contraction_lists.append(contraction_list_new)
            #print("contraction_list_new: " + str(contraction_list_new))

        N = len(new_field_lists)
    for i in range(N):
        field_list_new = new_field_lists[i]
        contraction_list_new = new_contraction_lists[i]
            
        term_new = term(field_list_new, contraction_list_new)
        term_list.append(term_new)
                
    return term_list

In [19]:
term_object = terms1[0]
print(term_object.get_field_symbols())
print(term_object.get_contractions())
product_rule_term_list = differentiate(term_object)
for item in product_rule_term_list:
    print("item.get_field_symbols(): " + str(item.get_field_symbols()))
    print("item.get_contractions(): " + str(item.get_contractions()))
    

['D', 'P', 'P', 'P', 'P']
[]
item.get_field_symbols(): ['D', 'D', 'P', 'P', 'P', 'P']
item.get_contractions(): [(0, 1)]
item.get_field_symbols(): ['D', 'P', 'D', 'P', 'P', 'P']
item.get_contractions(): [(0, 2)]
item.get_field_symbols(): ['D', 'P', 'P', 'D', 'P', 'P']
item.get_contractions(): [(0, 3)]
item.get_field_symbols(): ['D', 'P', 'P', 'P', 'D', 'P']
item.get_contractions(): [(0, 4)]


Check for equivalent terms. First write a function, term_reduce, to reduce terms to num_derivs_list and reduced contractions. Then write a function, terms_equiv, to determine whether two terms are equivalent. terms_equiv works by seeing whether the reduced, sorted versions of the two terms are equal. Reducing a term means converting its field list into a list of the number of derivatives acting on each field, and a list of pairs of field indices (each index from 1,...,num_fields) such that derivatives acting on them are contracted with each other. Then sorting it means sorting the list of numbers of derivatives on each field and making corresponding adjustments to the contraction indices. 

To see whether two terms are equivalent, one needs a way to reduce and sort terms; these functions are performed respectively by term_reduce and sort_reduced. 

In [20]:
def term_reduce(term_object):
    #INPUT
    #term1: term object
    #OUTPUT
    #num_derivs_list, contractions_reduced 
    #EXPLANATION
    #takes field list for term object and reduces to ordered (largest first) list of number of derivatives per
    #field. takes contraction list and reduces to contraction indicating which fields contracted derivatives act on.
    
    field_list = term_object.get_field_list()
    contraction_list = term_object.get_contractions()
    
    num_derivs_list = []
    contraction_list_reduced = []
    phi_indices = []
    deriv_count = 0
    
    #calculate num_derivs_list
    for i in range(len(field_list)):
        if field_list[i].get_symbol() == 'D':
            deriv_count +=1
        elif field_list[i].get_symbol() == 'P':
            num_derivs_list.append(deriv_count)
            phi_indices.append(i)
            deriv_count = 0
    
    #loop through contractions in contraction_list, calculate reduced contractions
    for i in range(len(contraction_list)):
        contraction_0 = contraction_list[i][0]
        contraction_1 = contraction_list[i][1]
        
        #if contracted index is less than term object index of phi, set index to number of phi field, from (0,...,N)
        for j in range(len(phi_indices)):    
            if contraction_0 < phi_indices[j]:
                contraction_reduced_0 = j
                break
        for j in range(len(phi_indices)): 
            if contraction_1 < phi_indices[j]:
                contraction_reduced_1 = j
                break
        
        contraction_reduced = (contraction_reduced_0, contraction_reduced_1)
        contraction_list_reduced.append(contraction_reduced)
        
    return num_derivs_list, contraction_list_reduced

In [21]:
for item in product_rule_term_list:
    print(item.get_field_symbols())
    print(item.get_contractions())
    num_derivs_list = term_reduce(item)[0]
    contraction_list_reduced = term_reduce(item)[1]
    print("num_derivs_list: " + str(num_derivs_list))
    print("contraction_list_reduced: " + str(contraction_list_reduced))
    print("")
    
    
    
'''
term_object = terms2[]
print(term_object.get_field_symbols())
print(term_object.get_contractions())
num_derivs_list, contraction_list_reduced = term_reduce(term_object)
print(num_derivs_list)
print(contraction_list_reduced)
'''

['D', 'D', 'P', 'P', 'P', 'P']
[(0, 1)]
num_derivs_list: [2, 0, 0, 0]
contraction_list_reduced: [(0, 0)]

['D', 'P', 'D', 'P', 'P', 'P']
[(0, 2)]
num_derivs_list: [1, 1, 0, 0]
contraction_list_reduced: [(0, 1)]

['D', 'P', 'P', 'D', 'P', 'P']
[(0, 3)]
num_derivs_list: [1, 0, 1, 0]
contraction_list_reduced: [(0, 2)]

['D', 'P', 'P', 'P', 'D', 'P']
[(0, 4)]
num_derivs_list: [1, 0, 0, 1]
contraction_list_reduced: [(0, 3)]



'\nterm_object = terms2[]\nprint(term_object.get_field_symbols())\nprint(term_object.get_contractions())\nnum_derivs_list, contraction_list_reduced = term_reduce(term_object)\nprint(num_derivs_list)\nprint(contraction_list_reduced)\n'

In [22]:
def sort_reduced(num_derivs_list, contraction_list_reduced):
    #INPUT
    #num_derivs_list: unsorted list of number of derivatives associated with each field
    #contraction_list_reduced: unsorted list of reduced contractions
    #OUTPUT
    #contractions_sorted: contraction list modified to reflect permutation to reach sorted num_derivs_list
    #EXPLANATION
    #sort num_derivs_list from largest to smallest, making the corresponding change to contractions in contraction_list. 
    #for now, use bubble sort. that is, go through num_derivs_list, if entry to the right is greater than entry to the left, interchange them, at the
    #same time making the corresponding change to contractions in contraction_list. keep doing this until there is a pass with no 
    #switches. 
    N_phi = len(num_derivs_list)
    num_derivs_list_sorted = num_derivs_list.copy()
    
    N_c = len(contraction_list_reduced)
    #print("contraction_list_reduced: " + str(contraction_list_reduced))
    #change 
    contraction_list_reduced_sorted = []
    for i in range(N_c): #initialize contractions_list_reduced_sorted to contractions_list_reduced, but as list of lists (mutable) rather than list of tuples (immutable)
        contraction_list_reduced_sorted.append([contraction_list_reduced[i][0], contraction_list_reduced[i][1]])
    
    #sort derivative and contraction lists
    is_sorted = 0
    while is_sorted==0:
        #print("contraction_list_reduced_sorted: " + str(contraction_list_reduced_sorted))
        is_sorted = 1
        for i in range(N_phi-1):
            #print("i: "+ str(i))
            if num_derivs_list_sorted[i] < num_derivs_list_sorted[i+1]:
                #print("i and i+1 out of order")
                is_sorted = 0
                temp = num_derivs_list_sorted[i]
                num_derivs_list_sorted[i] = num_derivs_list_sorted[i+1]
                num_derivs_list_sorted[i+1] = temp
                #make corresponding shift in contractions_list, if any.
                #check if there are any contraction indices equal to either of the two being interchanged; this involves
                #searching through contractions. loop over contractions and their components to see whether there are 
                #any equal to either of the indices being exchanged. for each index value equal to one of 
                #the exchanged indices, in each of the contractions containing that index value, replace it with
                #the index value with which it is exchanged. 
                
                #scan through reduced contractions to see whether any of contracted indices is equal to i; if so,
                #change them to i+1. 
                for j in range(N_c):
                    #print("j: " + str(j))
                    #find the two contracted indices 
                    
                    if contraction_list_reduced_sorted[j][0]==i:
                        contraction_list_reduced_sorted[j][0] = i+1
                    if contraction_list_reduced_sorted[j][1]==i:
                        contraction_list_reduced_sorted[j][1] = i+1
                        
                    if contraction_list_reduced_sorted[j][0]==i+1:
                        contraction_list_reduced_sorted[j][0] = i
                    if contraction_list_reduced_sorted[j][1]==i+1:
                        contraction_list_reduced_sorted[j][1] = i
                    
                    '''
                    if contraction_list_reduced_sorted[j][0] == i:
                        temp = 
                        contraction_list_reduced_sorted[j][0] = i+1 
                    elif contraction_list_reduced[j][1] == i:
                        contraction_list_reduced_sorted[j][1] = i+1
                    else:
                        pass

                    if contraction_list_reduced[j][0] == i+1:
                        contraction_list_reduced_sorted[j][0] = i 
                    elif contraction_list_reduced[j][1] == i+1:
                        contraction_list_reduced_sorted[j][1] = i
                    else:
                        pass
                    '''
        #print("num_derivs_list_sorted: " + str(num_derivs_list_sorted))
        #print("is_sorted: " + str(is_sorted))
                    
    return num_derivs_list_sorted, contraction_list_reduced_sorted           

In [23]:
#test sort_reduced
for item in product_rule_term_list:
    print(item.get_field_symbols())
    print(item.get_contractions())
    num_derivs_list, contraction_list_reduced = term_reduce(item)
    print("num_derivs_list: " + str(num_derivs_list))
    print("contraction_list_reduced: " + str(contraction_list_reduced))
    num_derivs_list_sorted, contraction_list_reduced_sorted = sort_reduced(num_derivs_list, contraction_list_reduced)
    print("num_derivs_list_sorted: " + str(num_derivs_list_sorted))
    print("contraction_list_reduced_sorted: " + str(contraction_list_reduced_sorted))
    print("")


['D', 'D', 'P', 'P', 'P', 'P']
[(0, 1)]
num_derivs_list: [2, 0, 0, 0]
contraction_list_reduced: [(0, 0)]
num_derivs_list_sorted: [2, 0, 0, 0]
contraction_list_reduced_sorted: [[0, 0]]

['D', 'P', 'D', 'P', 'P', 'P']
[(0, 2)]
num_derivs_list: [1, 1, 0, 0]
contraction_list_reduced: [(0, 1)]
num_derivs_list_sorted: [1, 1, 0, 0]
contraction_list_reduced_sorted: [[0, 1]]

['D', 'P', 'P', 'D', 'P', 'P']
[(0, 3)]
num_derivs_list: [1, 0, 1, 0]
contraction_list_reduced: [(0, 2)]
num_derivs_list_sorted: [1, 1, 0, 0]
contraction_list_reduced_sorted: [[0, 1]]

['D', 'P', 'P', 'P', 'D', 'P']
[(0, 4)]
num_derivs_list: [1, 0, 0, 1]
contraction_list_reduced: [(0, 3)]
num_derivs_list_sorted: [1, 1, 0, 0]
contraction_list_reduced_sorted: [[0, 1]]



Maybe easier to do something different - e.g., write a function testing whether two term objects are equivalent. This requires that their sorted num_derivs_list's and sorted contraction_list's be the same.  

In [24]:
def terms_equiv(term_object_1, term_object_2):
    #INPUT
    #term_object1: term_object
    #term_object2: term_object
    #OUTPUT
    #equiv: True if terms are equivalent, False if not. Equivalence is determined by whether sorted num_deriv_lists
    #of two terms are the same, and whether  contraction lists are equivalent relative to this num_deriv_list
    
    #Check that field_lists and contraction_lists of terms have the same length
    if len(term_object_1.get_field_list()) != len(term_object_2.get_field_list()) or len(term_object_1.get_contractions()) != len(term_object_2.get_contractions()):
        return False
    else:
        num_derivs_list_1, contraction_list_reduced_1 = term_reduce(term_object_1)
        num_derivs_list_2, contraction_list_reduced_2 = term_reduce(term_object_2)
        #print("num_derivs_list_1: " + str(num_derivs_list_1))
        #print("num_derivs_list_2: " + str(num_derivs_list_2))
        #print("contraction_list_reduced_1: " + str(contraction_list_reduced_1))
        #print("contraction_list_reduced_2: " + str(contraction_list_reduced_2))

        num_derivs_list_1_sorted, contraction_list_reduced_1_sorted = sort_reduced(num_derivs_list_1, contraction_list_reduced_1)
        num_derivs_list_2_sorted, contraction_list_reduced_2_sorted = sort_reduced(num_derivs_list_2, contraction_list_reduced_2)

        derivs_equiv = (num_derivs_list_1_sorted == num_derivs_list_2_sorted)
        #print("num_derivs_list_1_sorted: " + str(num_derivs_list_1_sorted))
        #print("num_derivs_list_2_sorted: " + str(num_derivs_list_2_sorted))
        #print("derivs_equiv: " + str(derivs_equiv))

        contractions_equiv = equiv_contraction_list(contraction_list_reduced_1_sorted, contraction_list_reduced_2_sorted, num_derivs_list_1_sorted) #can also use num_derivs_list_2_sorted, assuming they are equivalent as they should be
        #print("contraction_list_reduced_1_sorted: " + str(contraction_list_reduced_1_sorted))
        #print("contraction_list_reduced_2_sorted: " + str(contraction_list_reduced_2_sorted))
        #print("derivs_equiv: " + str(derivs_equiv))
        #print("contractions_equiv: " + str(contractions_equiv))
        equiv = derivs_equiv & contractions_equiv

        return equiv

In [25]:
for item in product_rule_term_list:
    print("item.get_field_symbols(): " + str(item.get_field_symbols()))
    print("item.get_contractions(): " + str(item.get_contractions()))

print("")
terms_equiv(product_rule_term_list[1], product_rule_term_list[2])

item.get_field_symbols(): ['D', 'D', 'P', 'P', 'P', 'P']
item.get_contractions(): [(0, 1)]
item.get_field_symbols(): ['D', 'P', 'D', 'P', 'P', 'P']
item.get_contractions(): [(0, 2)]
item.get_field_symbols(): ['D', 'P', 'P', 'D', 'P', 'P']
item.get_contractions(): [(0, 3)]
item.get_field_symbols(): ['D', 'P', 'P', 'P', 'D', 'P']
item.get_contractions(): [(0, 4)]



True

Test some more examples. 

In [26]:
D = field('D', 1, 'u')
P = field('P', 1)
term_object_2 = term([D, D, D, D, P, D, P, P, D, D, P], [(0, 1), (2, 3), (5, 8)])
print(term_object_2.get_field_symbols())
print(term_object_2.get_contractions())

product_rule_term_list_2 = differentiate(term_object_2)
for item in product_rule_term_list_2:
    print("item.get_field_symbols(): " + str(item.get_field_symbols()))
    print("item.get_contractions(): " + str(item.get_contractions()))

['D', 'D', 'D', 'D', 'P', 'D', 'P', 'P', 'D', 'D', 'P']
[(0, 1), (2, 3), (5, 8)]
item.get_field_symbols(): ['D', 'D', 'D', 'D', 'D', 'P', 'D', 'P', 'P', 'D', 'D', 'P']
item.get_contractions(): [(0, 1), (2, 3), (6, 9), (4, 10)]
item.get_field_symbols(): ['D', 'D', 'D', 'D', 'P', 'D', 'D', 'P', 'P', 'D', 'D', 'P']
item.get_contractions(): [(0, 1), (2, 3), (5, 9), (6, 10)]
item.get_field_symbols(): ['D', 'D', 'D', 'D', 'P', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
item.get_contractions(): [(0, 1), (2, 3), (5, 9), (7, 10)]
item.get_field_symbols(): ['D', 'D', 'D', 'D', 'P', 'D', 'P', 'P', 'D', 'D', 'D', 'P']
item.get_contractions(): [(0, 1), (2, 3), (5, 8), (9, 10)]


In [27]:
 terms_equiv(product_rule_term_list_2[0], product_rule_term_list_2[1])

False

Linear algebra: generate matrix of IBP and EOM constraints, and eliminate operators corresponding to pivot points in reduced row echelon form of this matrix.

In [30]:
'''
def generate_coefficient_vector(term_list, massDim, up_to = False):
    #INPUT
    #term_list: list of terms, should all be of same massDim 
    #massDim: int mass dimension
    #up_to: if True, generate coefficient vector corresponding to full set of operators up to, rather than only at,
    #massDim
    #OUTPUT
    #coefficient_vector: list of integers of length equal to total number of terms generated for massDim, 
    #indicating number of times each term
    
    all_terms = generate_fully_contracted(massDim) #list of all terms of mass dimension massDim 
    N = len(all_terms) #number of terms up to mass dimension massDim
    coefficient_vector = N*[0]
    
    for item in term_list:
        for n in range(N):
            print("item.get_field_symbols(): " + str(item.get_field_symbols()))
            print("item.get_contractions(): " + str(item.get_contractions()))
            print("all_terms[" + str(n) + "].get_field_symbols(): " + str(all_terms[n].get_field_symbols()))
            print("all_terms[" + str(n) + "].get_contractions(): " + str(all_terms[n].get_contractions()))
            print("terms_equiv(item, all_terms[n]): " + str(terms_equiv(item, all_terms[n])))
            if terms_equiv(item, all_terms[n]):
                coefficient_vector[n] += 1
    return coefficient_vector
'''
#DOUBLE CHECK - need to ensure that generate_fully_contracted(massDim) works properly for odd massDims - should
#return empty list
def generate_coefficient_vector(term_list, massDim, up_to=False):
    #INPUT
    #term_list: list of terms, should all be of same massDim 
    #massDim: int mass dimension
    #up_to: if True, generate coefficient vector corresponding to full set of operators up to, rather than only at,
    #massDim
    #OUTPUT
    #coefficient_vector: list of integers of length equal to total number of terms generated for massDim, 
    #indicating number of times each term
    
    if up_to == False:
        all_terms = generate_fully_contracted(massDim) #list of all terms of mass dimension massDim 
        N = len(all_terms) #number of terms of mass dimension massDim
        coefficient_vector = N*[0]

        for item in term_list:
            for n in range(N):
                #print("item.get_field_symbols(): " + str(item.get_field_symbols()))
                #print("item.get_contractions(): " + str(item.get_contractions()))
                #print("all_terms[" + str(n) + "].get_field_symbols(): " + str(all_terms[n].get_field_symbols()))
                #print("all_terms[" + str(n) + "].get_contractions(): " + str(all_terms[n].get_contractions()))
                #print("terms_equiv(item, all_terms[n]): " + str(terms_equiv(item, all_terms[n])))
                if terms_equiv(item, all_terms[n]):
                    coefficient_vector[n] += 1
        return coefficient_vector
    
    elif up_to == True:
        all_terms = []
        
        for m in range(massDim+1):
            all_terms += generate_fully_contracted(m)
        
        N = len(all_terms) #number of terms of mass dimension massDim
        coefficient_vector = N*[0]
        
        for item in term_list:
            for n in range(N):
                #print("")
                #print("item.get_field_symbols(): " + str(item.get_field_symbols()))
                #print("item.get_contractions(): " + str(item.get_contractions()))
                #print("all_terms[" + str(n) + "].get_field_symbols(): " + str(all_terms[n].get_field_symbols()))
                #print("all_terms[" + str(n) + "].get_contractions(): " + str(all_terms[n].get_contractions()))
                #print("terms_equiv(item, all_terms[n]): " + str(terms_equiv(item, all_terms[n])))
                if terms_equiv(item, all_terms[n]):
                    coefficient_vector[n] += 1
        return coefficient_vector
                  
    

In [31]:
massDim = 8
for item in generate_fully_contracted(massDim):
    print("item.get_field_symbols(): " + str(item.get_field_symbols()))
    print("item.get_contractions(): " + str(item.get_contractions()))
    print("")
    
for item in product_rule_term_list:
    print(item.get_field_symbols())
    print(item.get_contractions())

generate_coefficient_vector(product_rule_term_list, massDim, up_to=True)


item.get_field_symbols(): ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P']
item.get_contractions(): []

item.get_field_symbols(): ['D', 'P', 'D', 'P', 'P', 'P', 'P', 'P']
item.get_contractions(): [(0, 2)]

item.get_field_symbols(): ['D', 'D', 'P', 'P', 'P', 'P', 'P', 'P']
item.get_contractions(): [(0, 1)]

item.get_field_symbols(): ['D', 'P', 'D', 'P', 'D', 'P', 'D', 'P']
item.get_contractions(): [(0, 2), (4, 6)]

item.get_field_symbols(): ['D', 'D', 'P', 'D', 'P', 'D', 'P', 'P']
item.get_contractions(): [(0, 1), (3, 5)]

item.get_field_symbols(): ['D', 'D', 'P', 'D', 'P', 'D', 'P', 'P']
item.get_contractions(): [(0, 3), (1, 5)]

item.get_field_symbols(): ['D', 'D', 'D', 'P', 'D', 'P', 'P', 'P']
item.get_contractions(): [(0, 1), (2, 4)]

item.get_field_symbols(): ['D', 'D', 'P', 'D', 'D', 'P', 'P', 'P']
item.get_contractions(): [(0, 1), (3, 4)]

item.get_field_symbols(): ['D', 'D', 'P', 'D', 'D', 'P', 'P', 'P']
item.get_contractions(): [(0, 3), (1, 4)]

item.get_field_symbols(): ['D', 'D', 'D'

[0,
 0,
 0,
 0,
 0,
 3,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [32]:
def generate_IBP_matrix(massDim, up_to=False):
    #INPUT
    #massDim: mass dimension, int
    #OUTPUT
    #IBP_matrix: matrix (numpy array) of IBP constraints
    
    if up_to == False:
        IBP_matrix = []
        one_index_terms = generate_contracted_terms(massDim-1, num_free_indices=1)
        for item in one_index_terms:
            product_rule_term_list = differentiate(item)
            row = generate_coefficient_vector(product_rule_term_list, massDim, up_to=False)
            IBP_matrix.append(row)
        return IBP_matrix
    
    elif up_to == True:
        IBP_matrix = []
        for m in range(massDim+1):
            one_index_terms = generate_contracted_terms(m-1, num_free_indices=1)
            for item in one_index_terms:
                product_rule_term_list = differentiate(item)
                row = generate_coefficient_vector(product_rule_term_list, massDim, up_to=True)
                IBP_matrix.append(row)
        return IBP_matrix
            

massDim = 10
IBP_matrix = generate_IBP_matrix(massDim, up_to=True)

for row in IBP_matrix:
    print(row)


[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0

To find terms that can be eliminated, put IBP matrix into reduced row echelon form.

In [34]:
import sympy

IBP_matrix = sympy.Matrix(IBP_matrix)
print(IBP_matrix.rref())

(Matrix([
[0, 0, 1, 1, 0, 0,   0, 0, 0, 0,  0, 0, 0,   0, 0, 0, 0, 0,   0,    0,    0, 0, 0, 0, 0, 0,  0, 0, 0,   0, 0, 0, 0, 0,   0,     0,     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,    0, 0,    0,    0, 0, 0, 0, 0, 0, 0, 0, 0,  0],
[0, 0, 0, 0, 0, 1, 1/3, 0, 0, 0,  0, 0, 0,   0, 0, 0, 0, 0,   0,    0,    0, 0, 0, 0, 0, 0,  0, 0, 0,   0, 0, 0, 0, 0,   0,     0,     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,    0, 0,    0,    0, 0, 0, 0, 0, 0, 0, 0, 0,  0],
[0, 0, 0, 0, 0, 0,   0, 1, 0, 0,  1, 0, 0,   0, 0, 0, 0, 0,   0,    0,    0, 0, 0, 0, 0, 0,  0, 0, 0,   0, 0, 0, 0, 0,   0,     0,     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,    0, 0,    0,    0, 0, 0, 0, 0, 0, 0, 0, 0,  0],
[0, 0, 0, 0, 0, 0,   0, 0, 1, 0, -1, 0, 0,   0, 0, 0, 0, 0,   0,    0,    0, 0, 0, 0, 0, 0,  0, 0, 0,   0, 0, 0, 0, 0,   0,     0,     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,    0, 0,    0,    0, 0, 0, 0, 0, 0, 0, 0, 0,  0],
[0, 0, 0, 0, 0, 0,   0, 0, 0, 1, -1, 0, 0,   0

Can remove operators associated with indices (1, 3, 4, 5). Operators associated with other indices - 0, 2, 6 - span the space of operators of dimension 6. Let's remind ourselves what operators these indices represent:

In [35]:
all_terms = generate_fully_contracted(6)
N = len(all_terms)
for i in range(N):
    print(str(i) + ": " + str(all_terms[i].get_field_symbols()))
    print(str(i) + ": " + str(all_terms[i].get_contractions()))
    

0: ['P', 'P', 'P', 'P', 'P', 'P']
0: []
1: ['D', 'P', 'D', 'P', 'P', 'P']
1: [(0, 2)]
2: ['D', 'D', 'P', 'P', 'P', 'P']
2: [(0, 1)]
3: ['D', 'D', 'D', 'P', 'D', 'P']
3: [(0, 1), (2, 4)]
4: ['D', 'D', 'P', 'D', 'D', 'P']
4: [(0, 1), (3, 4)]
5: ['D', 'D', 'P', 'D', 'D', 'P']
5: [(0, 3), (1, 4)]
6: ['D', 'D', 'D', 'D', 'P', 'P']
6: [(0, 1), (2, 3)]


So the space of dim6 operators is spanned by the following operators: 

- 0: ['P', 'P', 'P', 'P', 'P', 'P']
0: [], 
- 2: ['D', 'D', 'P', 'P', 'P', 'P']
2: [(0, 1)]; 
- 6: ['D', 'D', 'D', 'D', 'P', 'P']
6: [(0, 1), (2, 3)]

In [36]:
massDim = 8
IBP_matrix_2 = generate_IBP_matrix(massDim)
IBP_matrix_2 = sympy.Matrix(IBP_matrix_2)
print(IBP_matrix_2)
print(IBP_matrix_2.rref())

Matrix([[0, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]])
(Matrix([
[0, 1, 1/5, 0, 0, 0, 0,   0,    0,    0, 0, 0, 0, 0, 0,  0],
[0, 0,   0, 1, 0, 0, 0,   0, -1/2,  1/6, 0, 0, 0, 0, 0,  0],
[0, 0,   0, 0, 1, 0, 0, 1/2,    0, -1/6, 0, 0, 0, 0, 0,  0],
[0, 0,   0, 0, 0, 1, 0,   0,  1/2, -1/6, 0, 0, 0, 0, 0,  0],
[0, 0,   0, 0, 0, 0, 1,   0,    0,  1/3, 0, 0, 0, 0, 0,  0],
[0, 0,   0, 0, 0, 0, 0,   0,    0,    0, 1, 0, 0, 0, 0,  1],
[0, 0,   0, 0, 0, 0, 0,   0,    0,    0, 0, 1, 0, 0, 0, -1],
[0, 

In [37]:
massDim = 10
IBP_matrix_3 = generate_IBP_matrix(massDim)
IBP_matrix_3 = sympy.Matrix(IBP_matrix_3)
print(IBP_matrix_3)
print(IBP_matrix_3.rref())

Matrix([[0, 7, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 4, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 4, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 5, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 

A few points to keep in mind about using EoM for reducing space of operators:

**EoMs couple operators of different mass dimension**. So EoMs can't be used to reduce basis of operators of a single mass dimension. So in applying in EoMs, we reduce the basis not of operators of a fixed mass dimension, but of operators *up to* a given mass dimension (assumed to be at least as big as the mass dimension of the operator with the largest mass dimension appearing in the EoMs).  

**EoM operators do not belong to set of Lagrangian operators** with phi<->-phi symmetry. So what does EoM imply for relationship between Lagrangian operators? Can perform various operations to both sides of the EoM so that operators are those appearing in Lagrangian - e.g. multiply both sides by P, then by either D^2 or P^2, then by either D^2 or P^2, then by either D^2 or P^2, etc. 

**Operator coefficients in EoM are not integers as they are in IBP relations. They are parameters of the renormalizable part of the Lagrangian**. Generate random coefficients for EOMs.


Plan: 

1) Write function to generate larger coefficient vector, generate_coefficient_vector_all, that includes all operators *up to* a given mass dimension, rather than just operators *of* a given mass dimension. 

2) Convert EoM relation, which does not contain operators occurring in the Lagrangian, into one that does. As default, use random coefficients (or maybe coeffs that are all one) - we are interested in which operators can be eliminated, not in the precise values of their coefficients. My guess is that it is only for special values of the coefficients (some set of measure zero) that the precise values of the coefficients may alter the operator eliminations that can be performed from what they are in the generic case. Maybe just try multiplying both sides of the EoM by P - this seems simplest. 

3) Concatenate all IBP and EOM relations up to a given mass dimension into a single matrix. Then put into reduced row echelon form (rref).

4) Eliminate all operators associated with pivot point indices.


## Generate EOM Relations

In [39]:
def generate_EOM_term_list():
    #EXPLANATION: EOM terms are not part of Lagrangian. To express EoM as a relationship among Lagrangian terms,
    #multiply all terms by P. This results in the following list of terms
    
    #define constituent fields
    D = field('D', 1, 'mu')
    P = field('P', 1)
    
    #define individual terms in EoM
    term1 = term([P, D, D, P], [(1, 2)])
    term2 = term([P, P], [])
    term3 = term([P, P, P, P], [])
    
    EOM_term_list = [term1, term2, term3]
    
    return EOM_term_list

def generate_EOM_coefficient_vector(massDim):
    
    EOM_term_list = generate_EOM_term_list()
    EOM_coefficient_vector = generate_coefficient_vector(EOM_term_list, massDim, up_to=True)
    
    return EOM_coefficient_vector

massDim = 8
EOM_coefficient_vector = generate_EOM_coefficient_vector(massDim)
EOM_coefficient_vector

[1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

## Combine IBP and EOM Relations into Single Matrix, and Eliminate Redundant Operators

In [23]:
def generate_IBP_EOM_matrix(massDim):
    IBP_matrix = generate_IBP_matrix(massDim, up_to=True)
    EOM_coefficient_vector = generate_EOM_coefficient_vector(massDim)
    
    IBP_EOM_matrix = IBP_matrix + [EOM_coefficient_vector]
    return IBP_EOM_matrix

IBP_EOM_matrix = sympy.Matrix(generate_IBP_EOM_matrix(massDim))

IBP_EOM_matrix
    

NameError: name 'sympy' is not defined

In [24]:

def generate_IBP_EOM_rref(massDim):
    #INPUT
    #massDim: mass dimension, int
    #OUTPUT
    #IBP_RREF: matrix (numpy array) of IBP constraints in reduced row echelon form (rref)
    #removed_indices: indices of operators that can be eliminated
    IBP_EOM_matrix = generate_IBP_EOM_matrix(massDim)
    IBP_EOM_matrix = sympy.Matrix(IBP_EOM_matrix)
    IBP_EOM_rref, removed_indices = IBP_EOM_matrix.rref()
    return IBP_EOM_rref, removed_indices

massDim = 8
IBP_EOM_rref, removed_indices = generate_IBP_EOM_rref(massDim)
print(removed_indices)
IBP_EOM_rref

NameError: name 'generate_IBP_matrix' is not defined

The list of operators to eliminate based on IBP and EOM are those corresponding to index values (0, 2, 5, 7, 8, 9, 12, 14, 15, 16, 17, 21, 22, 23, 24, 25). The remaining operators constitute a minimal basis for operators up to mass dimension 8. 

Compare indices to those generated without using EOM.

In [56]:
sympy.Matrix(generate_IBP_matrix(massDim, up_to=True)).rref()[1]

(2, 5, 7, 8, 9, 12, 14, 15, 16, 17, 21, 22, 23, 24, 25)

Note that without EOM, the operator corresponding to index 0, which was eliminated above, is not eliminated if we use only IBP.  

In [61]:
total_terms = 0
for m in range(massDim):
    total_terms += len(generate_fully_contracted(m))
total_terms

11

In [1]:
len((0, 2, 5, 7, 8, 9, 12, 14, 15, 16, 17, 21, 22, 23, 24, 25))

16